In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import tqdm
import pdb
from scipy.sparse import csr_matrix, linalg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import lightgbm as lgb

import random
import os

warnings.filterwarnings(action='ignore')

In [19]:
path= '/opt/ml/level1_bookratingprediction_recsys-level1-recsys-06/data/'

users = pd.read_csv(path+'users.csv')
books = pd.read_csv(path+'books.csv')
train_ratings = pd.read_csv(path+'train_ratings.csv')
test_ratings = pd.read_csv(path+'test_ratings.csv')
submit = pd.read_csv(path + 'sample_submission.csv')

print('users shape: ', users.shape)
print('books shape: ', books.shape)
print('train_ratings shape: ', train_ratings.shape)

users shape:  (68092, 3)
books shape:  (149570, 10)
train_ratings shape:  (306795, 3)


In [20]:
def rmse(real: list, predict: list) -> float:
    pred = np.array(predict)
    return np.sqrt(np.mean((real-pred) ** 2))

SEED = 42
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(SEED)


In [21]:
users['location_city'] = users['location'].apply(lambda x: x.split(',')[0])
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1])
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2])
users = users.drop(['location'], axis=1)

In [22]:
ratings = pd.concat([train_ratings, test_ratings]).reset_index(drop=True)
context_df = ratings.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author']], on='isbn', how='left')
train_df = train_ratings.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author']], on='isbn', how='left')
test_df = test_ratings.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author']], on='isbn', how='left')

In [23]:
loc_city2idx = {v:k for k,v in enumerate(context_df['location_city'].unique())}
loc_state2idx = {v:k for k,v in enumerate(context_df['location_state'].unique())}
loc_country2idx = {v:k for k,v in enumerate(context_df['location_country'].unique())}

In [24]:
train_df['location_city'] = train_df['location_city'].map(loc_city2idx)
train_df['location_state'] = train_df['location_state'].map(loc_state2idx)
train_df['location_country'] = train_df['location_country'].map(loc_country2idx)

test_df['location_city'] = test_df['location_city'].map(loc_city2idx)
test_df['location_state'] = test_df['location_state'].map(loc_state2idx)
test_df['location_country'] = test_df['location_country'].map(loc_country2idx)

In [25]:
def age_map(x: int) -> int:
    x = int(x)
    if x < 20:
        return 1
    elif x >= 20 and x < 30:
        return 2
    elif x >= 30 and x < 40:
        return 3
    elif x >= 40 and x < 50:
        return 4
    elif x >= 50 and x < 60:
        return 5
    else:
        return 6

In [26]:
train_df['age'] = train_df['age'].fillna(int(train_df['age'].mean()))
train_df['age'] = train_df['age'].apply(age_map)
test_df['age'] = test_df['age'].fillna(int(test_df['age'].mean()))
test_df['age'] = test_df['age'].apply(age_map)

In [27]:
category2idx = {v:k for k,v in enumerate(context_df['category'].unique())}
publisher2idx = {v:k for k,v in enumerate(context_df['publisher'].unique())}
language2idx = {v:k for k,v in enumerate(context_df['language'].unique())}
author2idx = {v:k for k,v in enumerate(context_df['book_author'].unique())}

In [28]:
train_df['category'] = train_df['category'].map(category2idx)
train_df['publisher'] = train_df['publisher'].map(publisher2idx)
train_df['language'] = train_df['language'].map(language2idx)
train_df['book_author'] = train_df['book_author'].map(author2idx)

test_df['category'] = test_df['category'].map(category2idx)
test_df['publisher'] = test_df['publisher'].map(publisher2idx)
test_df['language'] = test_df['language'].map(language2idx)
test_df['book_author'] = test_df['book_author'].map(author2idx)

In [29]:
train_df['id'] = train_df['user_id'].copy()
test_df['id'] = test_df['user_id'].copy()

train_df['bn'] = train_df['isbn'].copy()
test_df['bn'] = test_df['isbn'].copy()

In [30]:
for i in enumerate (list(train_df.columns)) :
    ca = i[1]
    train_df[ca] = train_df[ca].astype('str')
    test_df[ca] = test_df[ca].astype('str')

In [33]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306795 entries, 0 to 306794
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           306795 non-null  object
 1   isbn              306795 non-null  object
 2   rating            306795 non-null  object
 3   age               306795 non-null  object
 4   location_city     306795 non-null  object
 5   location_state    306795 non-null  object
 6   location_country  306795 non-null  object
 7   category          306795 non-null  object
 8   publisher         306795 non-null  object
 9   language          306795 non-null  object
 10  book_author       306795 non-null  object
 11  id                306795 non-null  object
 12  bn                306795 non-null  object
dtypes: object(13)
memory usage: 32.8+ MB


In [32]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

params_cat = {
        "task_type" : "GPU",
        "devices" : '0',
        "random_state": SEED,
        "learning_rate": 0.05,
        "n_estimators": 2000,
        "verbose" : 1,
        "objective" : "RMSE",
        "max_depth": 8,#trial.suggest_int("max_depth", 1, 16),
        "colsample_bylevel": 1,#trial.suggest_float("colsample_bylevel", 0.8, 1.0),
        #"subsample": 0.8, #trial.suggest_float("subsample", 0.3, 1.0), GPU 사용시 안될수도.
        "min_child_samples": 50, #trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": 300, #trial.suggest_int("max_bin", 200, 500),
        "cat_features" : list(train_df.drop(['rating','user_id','isbn'], axis =1).columns),
        "one_hot_max_size" : 10
}
train_x, val_x, train_y, val_y = train_test_split(train_df.drop(['rating','user_id','isbn'], axis = 1), train_df['rating'], test_size=0.2)

model = CatBoostRegressor(**params_cat)
model.fit(
    train_x,
    train_y,
    eval_set=[(val_x, val_y)],
    #early_stopping_rounds=10,
    verbose=False,
)

cat_pred = model.predict(val_x)
log_score = rmse(val_y, cat_pred)

TypeError: unsupported operand type(s) for -: 'str' and 'float'

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306795 entries, 0 to 306794
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           306795 non-null  object
 1   isbn              306795 non-null  object
 2   rating            306795 non-null  object
 3   age               306795 non-null  object
 4   location_city     306795 non-null  object
 5   location_state    306795 non-null  object
 6   location_country  306795 non-null  object
 7   category          306795 non-null  object
 8   publisher         306795 non-null  object
 9   language          306795 non-null  object
 10  book_author       306795 non-null  object
 11  id                306795 non-null  object
 12  bn                306795 non-null  object
dtypes: object(13)
memory usage: 32.8+ MB


In [ ]:
log_score

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003183 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 15800
[LightGBM] [Info] Number of data points in the train set: 245436, number of used features: 10
[LightGBM] [Info] Start training from score 7.068747
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 2.35352
[200]	valid_0's rmse: 2.3377
[300]	valid_0's rmse: 2.32986
[400]	valid_0's rmse: 2.32636
[500]	valid_0's rmse: 2.3213
[600]	valid_0's rmse: 2.3167
[700]	valid_0's rmse: 2.31337
[800]	valid_0's rmse: 2.31196
[900]	valid_0's rmse: 2.30973
[1000]	valid_0's rmse: 2.30709
[1100]	valid_0's rmse: 2.30593
[1200]	valid_0's rmse: 2.30512
[1300]	valid_0's rmse: 2.30456
[1400]	valid_0's rmse: 2.30326
[1500]	valid_0's rmse: 2.30175
[1600]	valid_0's rmse: 2.30064
[1700]	valid_0's rmse: 2.29969
[1800]	valid

In [ ]:

pred = model.predict(test_df.drop(['user_id', 'isbn', 'rating'],axis = 1))
test_df['rating'] = pred
submit['rating'] = test['rating']
submit.to_csv('../data/20221027_Catboost_Plueid_real.csv', index = False)

NameError: name 'model' is not defined